In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

# Load stock data from Yahoo Finance
ticker = 'AAPL'
data = yf.download(ticker, start='2020-01-01', end='2024-12-31')
data = data.dropna()
data.head()

In [ ]:
# Financial Ratios
total_assets = 150000
total_liabilities = 90000
net_income = 15000
revenue = 120000
shareholders_equity = total_assets - total_liabilities

current_ratio = 1.8
debt_to_equity = total_liabilities / shareholders_equity
net_profit_margin = net_income / revenue
roe = net_income / shareholders_equity

print(f'Debt to Equity: {debt_to_equity:.2f}, Net Margin: {net_profit_margin:.2%}, ROE: {roe:.2%}')

In [ ]:
# Time Series Returns & Cumulative Returns
data['Returns'] = data['Close'].pct_change()
data['Cumulative'] = (1 + data['Returns']).cumprod()
data[['Returns', 'Cumulative']].plot(figsize=(12,5), title='Returns and Cumulative Returns')

In [ ]:
# Portfolio Performance & Risk Metrics
tickers = ['AAPL', 'MSFT']
prices = yf.download(tickers, start='2020-01-01', end='2024-12-31')['Close']
returns = prices.pct_change().dropna()

weights = np.array([0.6, 0.4])
portfolio_return = np.sum(returns.mean() * weights) * 252
portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
sharpe_ratio = portfolio_return / portfolio_volatility

print(f'Annual Return: {portfolio_return:.2%}, Volatility: {portfolio_volatility:.2%}, Sharpe: {sharpe_ratio:.2f}')

In [ ]:
# CAPM & Beta Calculation
stock = yf.download('AAPL', start='2020-01-01', end='2024-12-31')['Close'].pct_change()
market = yf.download('^GSPC', start='2020-01-01', end='2024-12-31')['Close'].pct_change()

import statsmodels.api as sm
X = sm.add_constant(market.dropna())
y = stock.loc[X.index]
model = sm.OLS(y, X).fit()

print(model.summary())

In [ ]:
# Technical Analysis Indicators
data['SMA_50'] = data['Close'].rolling(50).mean()
data['SMA_200'] = data['Close'].rolling(200).mean()
data[['Close', 'SMA_50', 'SMA_200']].plot(figsize=(12,6), title='SMA Crossover')

In [ ]:
# Bond Price Calculation
def bond_price(face_value, coupon_rate, years, ytm, freq=1):
    periods = years * freq
    coupon = face_value * coupon_rate / freq
    return sum([coupon / (1 + ytm/freq)**t for t in range(1, periods+1)]) + face_value / (1 + ytm/freq)**periods

price = bond_price(1000, 0.05, 10, 0.04)
print(f'Bond Price: {price:.2f}')

In [ ]:
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(data['Returns'], kde=True)
plt.title('Histogram of Daily Returns')
plt.xlabel('Returns')

In [ ]:
# Value at Risk (VaR)
var_95 = np.percentile(data['Returns'].dropna(), 5)
print(f'95% Historical VaR: {var_95:.2%}')

In [ ]:
#Monte Carlo Simulation
simulations = 1000
T = 252
mu = returns["AAPL"].mean()
sigma = returns["AAPL"].std()
final_prices = []

for _ in range(simulations):
    daily_returns = np.random.normal(mu, sigma, T)
    price_series = 100 * (1 + pd.Series(daily_returns)).cumprod()
    final_prices.append(price_series.iloc[-1])

plt.hist(final_prices, bins=50)
plt.title("Simulated Final Prices")
plt.xlabel("Final Price")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()